In [1]:
# Importar biliotecas necessárias
import pandas as pd
import numpy as np
import zipfile
import requests
from io import BytesIO
import os

In [2]:
# Criar um diretório para armazenar conteúdo do ENADE
os.makedirs('./gabriel_enade2019', exist_ok=True)

In [3]:
url = "https://download.inep.gov.br/microdados/microdados_enem_2019.zip"

# Faz o download do conteúdo
filebytes = BytesIO(
    requests.get(url, stream=True).content
)

# Extrair o conteúdo do zipfile
myzip = zipfile.ZipFile(filebytes)
myzip.extractall("./gabriel_enade2019")

KeyboardInterrupt: 

# Trabalhando com os dados do ENEM usando desk

In [2]:
!ls

'ls' não é reconhecido como um comando interno
ou externo, um programa operável ou um arquivo em lotes.


In [3]:
!pip install dask

In [4]:
import dask.dataframe as dd

In [5]:
enem = dd.read_csv(
    "./gabriel_enade2019/DADOS/MICRODADOS_ENEM_2019.csv",
    sep = ";", decimal = ".", encoding = 'latin1'
)

In [6]:
enem = enem.loc[
    enem.SG_UF_RESIDENCIA == "MG"
]

In [7]:
enem.head(10)

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
1044,190001005671,2019,3170206,Uberlândia,31,MG,21,M,1,1,...,B,A,A,D,A,A,D,B,B,B
5234,190001009861,2019,3111705,Canaã,31,MG,20,M,1,1,...,A,A,A,B,A,A,C,A,A,A
16995,190001021622,2019,3104007,Araxá,31,MG,24,M,1,2,...,A,A,A,C,B,A,C,A,A,B
22008,190001026636,2019,3106200,Belo Horizonte,31,MG,33,M,1,1,...,B,A,B,B,A,A,D,A,D,B
22114,190001026742,2019,3162401,São João da Ponte,31,MG,23,M,1,3,...,A,A,A,B,B,A,C,A,B,A
41418,190001046046,2019,3124302,Espinosa,31,MG,22,M,1,3,...,B,A,A,C,B,A,E,A,B,B
41845,190001046473,2019,3169307,Três Corações,31,MG,19,F,1,3,...,B,A,A,B,B,B,E,A,B,B
48454,190001053082,2019,3106705,Betim,31,MG,20,M,1,3,...,B,A,A,B,A,A,D,A,B,B
51498,190001056126,2019,3143302,Montes Claros,31,MG,23,F,1,2,...,B,A,A,B,A,A,C,A,C,B
54094,190001058722,2019,3169307,Três Corações,31,MG,18,F,1,1,...,B,A,A,C,B,A,D,A,C,B


In [11]:
enem = enem.compute()

In [35]:
# Divide arquivo em mais partes por questões de memória filtrando por estado
chunk_size=1000000
batch_no=1
for chunk in pd.read_csv('./gabriel_enade2019/DADOS/MICRODADOS_ENEM_2019.csv', sep = ';', decimal = ',', chunksize=chunk_size, encoding = "ISO-8859-1"):
    chunk[chunk.SG_UF_RESIDENCIA == 'MG'].to_csv('./gabriel_enade2019/CHUNK/chunk'+str(batch_no)+'.csv', sep = ';', decimal = ',', encoding = "ISO-8859-1", index=False)
    batch_no+=1

In [36]:
# Cria um DataFrame único com todos os arquivos filtrados
columns= chunk.columns
enem = pd.DataFrame(columns = columns)
for i in range(1, batch_no):
    input = pd.read_csv('./gabriel_enade2019/CHUNK/chunk'+str(i)+'.csv', sep = ';', decimal = ',', encoding = "ISO-8859-1")
    enem = enem.append(input, ignore_index =True)

In [38]:
# Cria um arquivo CSV único com os dados filtrados 
enem.to_csv('./gabriel_enade2019/DADOS/MICRODADOS_ENEM_2019_MG.csv', sep = ';', decimal = ',', encoding = "ISO-8859-1", index=False)

In [46]:
enade = pd.read_csv(
    './gabriel_enade2019/DADOS/MICRODADOS_ENEM_2019_MG.csv',
    sep = ';', decimal = ',', encoding = "ISO-8859-1"
)

In [7]:

enem = pd.read_csv(
    "./gabriel_enade2019/DADOS/MICRODADOS_ENEM_2019.csv",
    sep = ";", decimal = ".", encoding = 'ISO-8859-1'
)

In [8]:
# 01) Qual é média de matemática de todos os alunos mineiros

enem.loc[
    (enem.SG_UF_RESIDENCIA == "MG")    
].NU_NOTA_MT.describe()

count    393872.000000
mean        546.796208
std         115.072018
min           0.000000
25%         451.800000
50%         531.400000
75%         629.800000
max         985.500000
Name: NU_NOTA_MT, dtype: float64

In [6]:
# 02) Qual é  média da nota em Linguagens e Códigos de todos os alunos mineiros

enem.loc[
    (enem.SG_UF_RESIDENCIA == "MG")    
].NU_NOTA_LC.describe()

count    414346.000000
mean        531.215550
std          61.324732
min           0.000000
25%         497.400000
50%         536.500000
75%         572.900000
max         770.500000
Name: NU_NOTA_LC, dtype: float64

In [12]:
# 03) Qual é a média da nota em Ciências Humanas dos alunos do sexo FEMININO mineiros
enem.loc[
    (enem.SG_UF_RESIDENCIA == "MG")&
    (enem.TP_SEXO == "F")
].NU_NOTA_CH.describe()

count    251816.000000
mean        515.127098
std          81.019828
min           0.000000
25%         458.200000
50%         519.300000
75%         572.600000
max         795.000000
Name: NU_NOTA_CH, dtype: float64

In [19]:
# 04) Qual é a média da nota em Ciências Humanas dos alunos do sexo MASCULINO
enem.loc[
    (enem.SG_UF_RESIDENCIA == "MG")&
    (enem.TP_SEXO == "M")
].NU_NOTA_CH.describe()

count    162530.000000
mean        529.698270
std          85.803672
min           0.000000
25%         470.200000
50%         538.500000
75%         591.500000
max         835.100000
Name: NU_NOTA_CH, dtype: float64

In [20]:
# 05) Qual é a média da nota em Matemática dos alunos do sexo FEMININO que moram na cidade de Montes Claros
enem.loc[
    (enem.SG_UF_RESIDENCIA == "MG")&
    (enem.TP_SEXO == "F")&
    (enem.NO_MUNICIPIO_RESIDENCIA == "Montes Claros")    
].NU_NOTA_MT.describe()

count    7699.000000
mean      525.477672
std       107.081759
min       359.300000
25%       438.750000
50%       505.400000
75%       601.000000
max       913.700000
Name: NU_NOTA_MT, dtype: float64

In [27]:
# 06) Qual é a média da nota em Matemática dos alunos do município de Sabará que possuem TV por assinatura na residência
enem.loc[
    (enem.SG_UF_RESIDENCIA == "MG")&
    (enem.NO_MUNICIPIO_RESIDENCIA == "Sabará")&  
    (enem.Q021 == "B")
].NU_NOTA_MT.describe()

count    704.000000
mean     543.292756
std      115.270483
min      359.100000
25%      444.125000
50%      521.850000
75%      622.425000
max      924.700000
Name: NU_NOTA_MT, dtype: float64

In [25]:
# 07) Qual é a média da nota em Ciências Humanas dos alunos mineiros que possuem 2 fornos micro-ondas em casa
enem.loc[
    (enem.SG_UF_RESIDENCIA == "MG")&
    (enem.Q016 == "C")
].NU_NOTA_CH.describe()

count    2205.000000
mean      557.276599
std        90.127265
min         0.000000
25%       498.000000
50%       572.500000
75%       622.200000
max       775.500000
Name: NU_NOTA_CH, dtype: float64

In [26]:
# 08) Qual é a média da nota em Matemática dos alunos mineiros cuja mãe completou a pós-graduação
enem.loc[
    (enem.SG_UF_RESIDENCIA == "MG")&
    (enem.Q002 == "G")
].NU_NOTA_MT.describe()

count    40937.000000
mean       620.007062
std        122.608027
min          0.000000
25%        525.600000
50%        629.200000
75%        710.200000
max        985.500000
Name: NU_NOTA_MT, dtype: float64

In [40]:
# 09) Qual é a média da nota em Matemática dos alunos de Belo Horizonte e de Conselheiro Lafaiete
enem.loc[
    (enem.SG_UF_RESIDENCIA == "MG")&
    (enem.NO_MUNICIPIO_RESIDENCIA.isin(["Belo Horizonte","Conselheiro Lafaiete"]))
].NU_NOTA_MT.describe()

count    59238.000000
mean       578.039227
std        128.510689
min          0.000000
25%        469.700000
50%        570.500000
75%        674.200000
max        985.500000
Name: NU_NOTA_MT, dtype: float64

In [41]:
# 10) Qual é a média da nota em Ciências Humanas dos alunos mineiros que moram sozinhos
enem.loc[
    (enem.SG_UF_RESIDENCIA == "MG")&
    (enem.Q005 == 1)
].NU_NOTA_CH.describe()

count    10131.000000
mean       534.457339
std         86.729627
min          0.000000
25%        478.950000
50%        543.200000
75%        595.300000
max        786.600000
Name: NU_NOTA_CH, dtype: float64

In [43]:
# 11) Qual é a média da nota em Ciências Humanas dos alunos mineiros cujo pai completou a Pós-Graduação e possuem renda familiar entre R$ 8.982,01 e R$ 9.980,00
enem.loc[
    (enem.SG_UF_RESIDENCIA == "MG")&
    (enem.Q001 == "G")&
    (enem.Q006 == "M")
].NU_NOTA_CH.describe()

count    1118.000000
mean      586.723166
std        75.330683
min       319.300000
25%       550.425000
50%       598.450000
75%       637.525000
max       784.000000
Name: NU_NOTA_CH, dtype: float64

In [48]:
# 12) Qual é a média da nota em Matemática dos alunos do sexo FEMININO que moram em Lavras e escolhem "Espanhol" como língua estrangeira
enem.loc[
    (enem.SG_UF_RESIDENCIA == "MG")&
    (enem.TP_SEXO == "F")&
    (enem.NO_MUNICIPIO_RESIDENCIA == "Lavras")&
    (enem.TP_LINGUA == 1)
].NU_NOTA_MT.describe()

count    894.000000
mean     510.809508
std       94.541685
min      359.900000
25%      435.175000
50%      492.200000
75%      570.350000
max      870.200000
Name: NU_NOTA_MT, dtype: float64

In [49]:
# 13) Qual é a média da nota em Matemática dos alunos do sexo MASCULINO que moram em Ouro Preto
enem.loc[
    (enem.SG_UF_RESIDENCIA == "MG")&
    (enem.TP_SEXO == "M")&
    (enem.NO_MUNICIPIO_RESIDENCIA == "Ouro Preto")
].NU_NOTA_MT.describe()

count    1230.000000
mean      555.083252
std       114.936273
min       359.000000
25%       461.250000
50%       540.500000
75%       638.650000
max       903.900000
Name: NU_NOTA_MT, dtype: float64

In [50]:
# 14) Qual é a média da nota em Ciências Humanas dos alunos surdos
enem.loc[
    (enem.SG_UF_RESIDENCIA == "MG")&
    (enem.IN_SURDEZ == 1)
].NU_NOTA_CH.describe()

count    108.000000
mean     435.387963
std       61.942581
min      330.500000
25%      395.100000
50%      426.350000
75%      471.250000
max      616.500000
Name: NU_NOTA_CH, dtype: float64

In [54]:
# 15) Qual é a média da nota em Matemática dos alunos do sexo FEMININO, que moram em Belo Horizonte, Sabará, Nova Lima e Betim e possuem dislexia
enem.loc[
    (enem.SG_UF_RESIDENCIA == "MG")&
    (enem.TP_SEXO == "F")&
    (enem.NO_MUNICIPIO_RESIDENCIA.isin(["Belo Horizonte","Sabará","Nova Lima","Betim"]))&
    (enem.IN_DISLEXIA == 1)
].NU_NOTA_MT.describe()

count     31.000000
mean     582.193548
std      131.003641
min      363.600000
25%      474.250000
50%      604.400000
75%      672.750000
max      789.100000
Name: NU_NOTA_MT, dtype: float64